In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

In [3]:
# Read in the employment data
empl = pd.read_csv("../data/CV_EMPL.csv", na_values=["***", ".", "NA"])
empl['DATE'] = empl['DATE'].str.pad(width=6, side="left", fillchar="0")
empl["Total"] = empl.loc[:, "Agriculture, Forestry, Fishing and Hunting":].sum(axis=1)
empl['DATE'] = pd.to_datetime(empl.DATE, format='%y-%b')
empl["City"] = empl["City"].str.lower()
empl = empl.fillna(0)

for col in empl.columns:
    print(col)
    
empl.head()

DATE
City
Agriculture, Forestry, Fishing and Hunting
Utilities
Construction
Manufacturing
Wholesale Trade
Retail Trade
Transportation and Warehousing
Information
Finance and Insurance
Real Estate and Rental and Leasing
Professional, Scientific, and Technical Services
Management of Companies and Enterprises
Administrative and Support and Waste Management and Remediation Services
Educational Services
Health Care and Social Assistance
Arts, Entertainment, and Recreation
Accommodation and Food Services
Other Services (except Public Administration)
Federal Government
Local Government
State Government
Not Elsewhere Classified
Mining, Quarrying, and Oil and Gas Extraction
Total


,DATE,City,"Agriculture, Forestry, Fishing and Hunting",Utilities,Construction,Manufacturing,Wholesale Trade,Retail Trade,Transportation and Warehousing,Information,...,Health Care and Social Assistance,"Arts, Entertainment, and Recreation",Accommodation and Food Services,Other Services (except Public Administration),Federal Government,Local Government,State Government,Not Elsewhere Classified,"Mining, Quarrying, and Oil and Gas Extraction",Total
0,2005-01-01,cathedral city,0.0,0.0,971.0,79.0,106.0,1941.0,39.0,137.0,...,530.0,217.0,967.0,543.0,0.0,940.0,0.0,0.0,0.0,7401.0
1,2005-02-01,cathedral city,0.0,0.0,981.0,68.0,108.0,1956.0,39.0,143.0,...,517.0,224.0,990.0,547.0,0.0,941.0,0.0,0.0,0.0,7430.0
2,2005-03-01,cathedral city,0.0,0.0,1032.0,78.0,111.0,1947.0,41.0,142.0,...,513.0,236.0,1000.0,561.0,0.0,953.0,0.0,0.0,0.0,7529.0
3,2005-04-01,cathedral city,0.0,0.0,988.0,86.0,154.0,1936.0,41.0,140.0,...,523.0,234.0,1023.0,546.0,0.0,906.0,0.0,0.0,0.0,7373.0
4,2005-05-01,cathedral city,0.0,0.0,1028.0,88.0,154.0,1953.0,39.0,135.0,...,529.0,245.0,1022.0,552.0,0.0,910.0,0.0,0.0,0.0,7412.0


# Plots
We are going to make the change bar plots. This will require us to consolidate the industries:

In [4]:
def consolidate_industries(df: pd.DataFrame):
    """Returns a dataframe that has the 12 consolidated industries we want"""
    df["Logistics"] = df["Wholesale Trade"] + df["Transportation and Warehousing"] + df["Utilities"]
    df["FIRE"] = df["Finance and Insurance"] + df["Real Estate and Rental and Leasing"]
    df["Professional and Business Services"] = df["Professional, Scientific, and Technical Services"] + df["Management of Companies and Enterprises"] + df["Administrative and Support and Waste Management and Remediation Services"]
    df["Leisure and Hospitality"] = df["Arts, Entertainment, and Recreation"] + df["Accommodation and Food Services"]
    df["Education and Health Services"] = df["Educational Services"] + df["Health Care and Social Assistance"]
    df["Government"] = df["Federal Government"] + df["State Government"] + df["Local Government"]
    df["Mining and Natural Resources"] = df["Mining, Quarrying, and Oil and Gas Extraction"] + df["Agriculture, Forestry, Fishing and Hunting"]
    
    cols_to_select = ["DATE", "City", "Logistics", "Construction", "Manufacturing",
                      "Retail Trade", "Information", "FIRE",
                      "Professional and Business Services", "Leisure and Hospitality",
                      "Education and Health Services", "Government",
                      "Mining and Natural Resources", "Total"]
    
    res = df[cols_to_select]
    return res

empl_c = consolidate_industries(empl)
empl_c.head()

,DATE,City,Logistics,Construction,Manufacturing,Retail Trade,Information,FIRE,Professional and Business Services,Leisure and Hospitality,Education and Health Services,Government,Mining and Natural Resources,Total
0,2005-01-01,cathedral city,145.0,971.0,79.0,1941.0,137.0,357.0,553.0,1184.0,551.0,940.0,0.0,7401.0
1,2005-02-01,cathedral city,147.0,981.0,68.0,1956.0,143.0,365.0,537.0,1214.0,531.0,941.0,0.0,7430.0
2,2005-03-01,cathedral city,152.0,1032.0,78.0,1947.0,142.0,361.0,533.0,1236.0,534.0,953.0,0.0,7529.0
3,2005-04-01,cathedral city,195.0,988.0,86.0,1936.0,140.0,317.0,459.0,1257.0,543.0,906.0,0.0,7373.0
4,2005-05-01,cathedral city,193.0,1028.0,88.0,1953.0,135.0,311.0,429.0,1267.0,546.0,910.0,0.0,7412.0


In [11]:
def peak_to_trough_empl(df: pd.DataFrame, city: str = "Rancho Mirage"):
    # Filter for the city
    city = df[df["City"] == city.lower()]
    
    # Set the index to be a datetime index and get rid of empty rows
    city = city.set_index("DATE")
    city = city[city["Total"] != 0]
    
    # Get the present value for each industry and note which 
    now = city.iloc[-1, :]
    
    # Get rid of any industries that have no present workers
    cols_to_drop = []
    for idx, item in now.iteritems():
        try:
            item = int(item)
            if item == 0:
                cols_to_drop.append(idx)
        except ValueError:
            continue
    city = city.drop(columns=cols_to_drop + ["Total"])
    industryCols = city.loc[:, "Logistics":].columns
    
    # Get dataframes to calculate the max and min for each industry
    max_frame = city.loc["2013-01-01":"2020-03-01", :]
    min_frame = city.loc["2020-03-01":, :]
    
    maxes = []
    mins = []
    now_vals = []
    for col in industryCols:
        maxes.append(max_frame[col].max())
        mins.append(min_frame[col].min())
        now_vals.append(now[col])
    
    graph_df = pd.DataFrame({
        "INDUSTRY": industryCols,
        "MAX": maxes,
        "MIN": mins,
        "NOW": now_vals
    })
    
    # Add columns to graph_df for the plot
    remaining = []
    recovery = []
    netgain = []
    
    for idx, item in graph_df.iterrows():
        if item["NOW"] < item["MAX"]:
            remaining.append(item["NOW"] - item["MAX"])
            netgain.append(0)
            recovery.append(item["MIN"] - item["NOW"])
        else:
            remaining.append(0)
            netgain.append(item["NOW"] - item["MAX"])
            recovery.append(item["MIN"] - item["MAX"])
    
    graph_df["remaining"] = remaining
    graph_df["recovery"] = recovery
    graph_df["netgain"] = netgain
    
    graph_df["remaining_perc"] = graph_df["remaining"] / graph_df["MAX"]
    graph_df["recovery_perc"] = graph_df["recovery"] / graph_df["MAX"]
    graph_df["netgain_perc"] = graph_df["netgain"] / graph_df["MAX"]
    
    fig = px.bar(graph_df, x="INDUSTRY",
                 y=["remaining", "recovery", "netgain"],
                 title=f"Peak-to-trough employment (Absolute Changes)")
    fig.show()
    
    fig2 = px.bar(graph_df, x="INDUSTRY",
                 y=["remaining_perc", "recovery_perc", "netgain_perc"],
                 title=f"Peak-to-trough employment (Percent Changes)")
    
    fig2.show()
    
    return graph_df
    

    
peak_to_trough_empl(empl_c)

,INDUSTRY,MAX,MIN,NOW,remaining,recovery,netgain,remaining_perc,recovery_perc,netgain_perc
0,Logistics,124.0,42.0,42.0,-82.0,0.0,0.0,-0.661290,0.000000,0.000000
1,Construction,239.0,94.0,110.0,-129.0,-16.0,0.0,-0.539749,-0.066946,0.000000
2,Manufacturing,77.0,0.0,49.0,-28.0,-49.0,0.0,-0.363636,-0.636364,0.000000
3,Retail Trade,1203.0,893.0,1001.0,-202.0,-108.0,0.0,-0.167914,-0.089776,0.000000
4,Information,171.0,12.0,18.0,-153.0,-6.0,0.0,-0.894737,-0.035088,0.000000
5,FIRE,407.0,335.0,341.0,-66.0,-6.0,0.0,-0.162162,-0.014742,0.000000
6,Professional and Business Services,837.0,524.0,644.0,-193.0,-120.0,0.0,-0.230585,-0.143369,0.000000
7,Leisure and Hospitality,6979.0,1483.0,2297.0,-4682.0,-814.0,0.0,-0.670870,-0.116636,0.000000
8,Education and Health Services,6091.0,5740.0,6262.0,0.0,-351.0,171.0,0.000000,-0.057626,0.028074
